### Guaranteed Scorer vs a Probabilistic Shooter
Prepared by David L. Parks  
December 22, 2020

Suppose that your favorite NBA team calls to get your opinion on their next draft pick. Assuming that you don't screen your calls and actually answer the phone, the options are:

1) Player 1: a 51% field goal shooter guaranteed to take 20 shots per game  
or  
2) Player 2: guaranteed to score 20 points per game  

If you are like most people, after a few seconds of thought you would instinctively pick Player 2. Scoring 20 guaranteed points per game is tough to beat. In fact, if Player 2 did this throughout his entire career, he would be number 70 on the points per game all-time leaderboard according to basketball-reference.com[1](https://www.basketball-reference.com/leaders/pts_per_g_career.html). However, if Player 2 squared off against Player 1, Player 1 would win more games, all else being equal. And winning, not scoring, is more important to the team - your instincts failed you.  

To see how this is possible, let's look at both a statistical solution and a simulation.

In [1]:
# import modules
import numpy as np
import scipy.stats as ss

We'll use the Binomial distribution for the statistical solution[2](https://en.wikipedia.org/wiki/Binomial_distribution). The binomial distribution with parameters n and p is the discrete probability distribution of the number of successes in a sequence of n independent experiments, like flipping a coin multiple times . For example, if a balanced coin was flipped 10 times, n = 10 and p = 0.5.

In [2]:
# set the number of shots taken by Player 1 (n) and the probability of success (p)
shots = 20
probability = 0.51

Using the cumulative distribution function for the binomial distribution, we compute the probability that Player 1 makes at least 10 shots. Subtracting this probability from 1 gives us the probability that Player 1 beats Player 2 (i.e. - Player 1 makes 11 or more shots). Next, we compute the probability that Player 1 makes at least 9 shots. This gives us the probability that Player 1 loses to Player 2. Lastly, we compute the probability that Player 1 makes exactly 10 shots. This gives us the probability that they tie.

In [3]:
# compute the probability that Player 1 makes at least 10 shots
p10 = np.round(ss.binom.cdf(10, shots, probability), 2)

# compute the probability that Player 1 makes at least 9 shots
p09 = np.round(ss.binom.cdf(9, shots, probability), 2)

# compute the probability that Player 1 makes exactly 10 shots
e10 = p10 - p09

In [4]:
print("Percent of time that Player 1 wins: {}%".format(np.round((1 - p10) * 100, 2)))
print("Percent of time that Player 2 wins: {}%".format(np.round(p09 * 100, 2)))
print("Percent of time that they tie: {}%".format(np.round(e10 * 100, 2)))

Percent of time that Player 1 wins: 45.0%
Percent of time that Player 2 wins: 38.0%
Percent of time that they tie: 17.0%


If they played an infinite number of games, Player 1 would win about 45% of the time, lose 38% of the time, and tie 17% of the time. If this still doesn't seem possible, let's conduct a simulation to confirm the previous results.

In [5]:
# define a function that computes the number of wins, losses,
# and ties for Player 1 and Player 2
def simulation(games, shots, p1_shot_probability, p2_points):
    # instantiate a random number generator and set the seed
    # for reproducibility
    rng = np.random.default_rng(38)
    
    # create zero arrays to capture the results
    p1_wins = np.zeros(games)
    p2_wins = np.zeros(games)
    ties = np.zeros(games)
    
    # iterate through the number of games
    for game in range(games):
        # set Player 1's initial points
        p1_points = 0
        
        # simulate Player 1 shooting
        for i in range(shots):
            if rng.random() < p1_shot_probability:
                p1_points += 2
        
        # determine who won
        if p1_points > p2_points:
            p1_wins[game] = 1
        elif p1_points == p2_points:
            ties[game] = 1
        elif p1_points < p2_points:
            p2_wins[game] = 1
            
    totals = np.array((np.sum(p1_wins), np.sum(p2_wins), np.sum(ties)))
    percentages = (totals / games) * 100
    return np.round(percentages, 2)

Simulate playing 100 games.

In [6]:
# set the parameters
games = 100
shots = 20
p1_shot_prob = 0.51
p2_points = 20

# run the simulation
results = simulation(games, shots, p1_shot_prob, p2_points)

# view the outcomes
print("Player 1 won {}% of the games.".format(results[0]))
print("Player 2 won {}% of the games.".format(results[1]))
print("They tied {}% of the games.".format(results[2]))

Player 1 won 41.0% of the games.
Player 2 won 40.0% of the games.
They tied 19.0% of the games.


We see from the results that Player 1 slighly edges out Player 2 after 100 games. But as we increase the number of games, we find that the percentages converge toward the percentages we found using the statistical solution.

Simulate playing 10,000 games.

In [7]:
# set the parameters
games = 10000
shots = 20
p1_shot_prob = 0.51
p2_points = 20

# run the simulation
results = simulation(games, shots, p1_shot_prob, p2_points)

# view the outcomes
print("Player 1 won {}% of the games.".format(results[0]))
print("Player 2 won {}% of the games.".format(results[1]))
print("They tied {}% of the games.".format(results[2]))

Player 1 won 44.23% of the games.
Player 2 won 38.06% of the games.
They tied 17.71% of the games.


Simulate playing 100,000 games.

In [8]:
# set the parameters
games = 100000
shots = 20
p1_shot_prob = 0.51
p2_points = 20

# run the simulation
results = simulation(games, shots, p1_shot_prob, p2_points)

# view the outcomes
print("Player 1 won {}% of the games.".format(results[0]))
print("Player 2 won {}% of the games.".format(results[1]))
print("They tied {}% of the games.".format(results[2]))

Player 1 won 45.04% of the games.
Player 2 won 37.46% of the games.
They tied 17.5% of the games.


Although not an exact match, these percentages are pretty close to the statistical solution.  

From the results above, we see that if winning is the primary goal, Player 1 is the better choice. This notebook illustrates the pitfalls associated with quick, intuitive decision-making and highlights the importance of understanding the customer's primary goal.